# ETH Data Preprocessing

This notebook demonstrates the process of loading and preprocessing Ethereum (ETH) daily price data from a CSV file. The primary goal is to prepare the data for further analysis or machine learning tasks.

# Table of Contents
- [Import Library](#import-library)
- [Load and Process Data Function](#load-and-process-data-function)

## Import Library

In [1]:
import pandas as pd

## Load and Process Data Function

> The `load_and_process_data` function reads Ethereum (ETH) daily price data from a given CSV file, renames and formats columns, sorts the data chronologically, drops unnecessary columns, and saves the processed data to a new CSV file named 'ohlcv_daily.csv', while returning the resulting Pandas DataFrame.

In [2]:
def load_and_process_data(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path, header=1)

    # Rename columns
    df.rename(columns={'date': 'trading_point', 'Volume USD': 'volume'}, inplace=True)

    # Convert 'trading_point' to datetime and remove timestamp
    df['trading_point'] = pd.to_datetime(df['trading_point']).dt.strftime('%Y-%m-%d')

    # Sort the DataFrame by 'trading_point'
    df.sort_values(by='trading_point', inplace=True)

    # Drop unnecessary columns
    df.drop(columns=['unix', 'symbol', 'Volume ETH'], inplace=True)

    # Save the DataFrame to a new CSV file
    df.to_csv('ohlcv_daily.csv', index=False)

    return df

In [10]:
eth_daily = load_and_process_data('./Bitstamp_ETHUSD_d.csv')
eth_daily.to_csv('ohlcv_daily.csv', index=False)
eth_hour = load_and_process_data('./Bitstamp_ETHUSD_1h.csv')
eth_hour.to_csv('ohlcv_hour.csv', index=False)

In [11]:
eth_daily.head()

,trading_point,open,high,low,close,volume
2254,2017-11-09,322.15,331.70,317.29,321.32,13118.15
2253,2017-11-10,321.32,325.61,291.00,298.66,34411.65
2252,2017-11-11,298.66,322.19,298.00,315.76,23836.74
2251,2017-11-12,315.76,318.98,300.11,306.10,48200.80
2250,2017-11-13,306.10,320.69,305.91,316.14,36613.50


In [12]:
eth_daily.shape

(2255, 6)

In [13]:
eth_hour.head()

,trading_point,open,high,low,close,volume
49602,2018-05-15,732.10,736.67,730.94,733.00,1043.91
49585,2018-05-15,700.99,710.15,699.05,707.38,255.89
49586,2018-05-15,707.84,711.29,695.74,700.99,1029.83
49587,2018-05-15,712.48,714.44,707.39,707.84,156.07
49588,2018-05-15,713.00,714.99,709.41,712.48,418.57


In [14]:
eth_hour.shape

(49603, 6)